In [1]:
import sys
!{sys.executable} -m pip install countryinfo

In [2]:
import pandas as pd
import numpy as np
from countryinfo import CountryInfo

In [3]:
df = pd.read_excel("aids.xlsx", 0, header=None, index_col=0)
df = df.fillna(method='ffill', axis=1) 
df.columns=pd.MultiIndex.from_arrays(df[:2].values) 
df.index.name = 'countries'
df = df[2:]

In [4]:
df.replace('-', np.NaN, inplace=True)
df.dropna(axis = 0, how = 'all', inplace=True)
df.replace('<', '', regex=True, inplace=True)
df = df.astype(np.float64)

In [5]:
df.columns = df.columns.to_flat_index()
df.columns = ['_'.join(col) for col in df.columns.values]

to_drop = [
    'HIV incidence per 1,000 uninfected population_Adolescents 10–19', 
    'AIDS-related mortality per 100,000 population_Adolescents 10–19', 
    'Number of children living with HIV_Adolescents 10–19'
]

df.drop(to_drop, axis=1, inplace=True) 

In [6]:
new_names = {
    "Côte d'Ivoire": "ivory coast", 
    "Iran (Islamic Republic of)": "Iran", 
    "Venezuela (Bolivarian Republic of)": "Venezuela", 
    "Bolivia (Plurinational State of)": "Bolivia",
    "Viet Nam": "Vietnam",
    "Eswatini": "swaziland",
    "Congo": "republic of the congo",
    "Gambia": "the gambia"
}

df.rename(index=new_names, inplace=True)

In [7]:
def get_regions_from_countries(countries):
    regions, available_countries = {}, CountryInfo().all()
    for country in countries:
        sub_region = CountryInfo(country).info()["subregion"]
        regions[country] = sub_region
    return regions

In [8]:
regions_countriers = get_regions_from_countries(df.index)

In [9]:
def get_sub_df(df, start):
    sub_df = df.iloc[:, start:start+3]
    sub_df["regions"] = pd.Series(regions_countriers)
    sub_df.columns = ["children_0-14", "girls_10–19", "boys_10–19", "regions"]
    return sub_df

In [10]:
# hiv incidence per 1000 uninfected population
hiv_incidence_df = get_sub_df(df, 0)
hiv_incidence_df.to_csv("hiv_incidence.csv", index_label="countries")

In [11]:
# AIDS-related mortality per 100,000 population
aids_mortality_df = get_sub_df(df, 3)
aids_mortality_df.to_csv("aids_mortality_df.csv", index_label="countries")

In [12]:
# Number of children living with HIV
children_living_hiv_df = get_sub_df(df, 6)
children_living_hiv_df.to_csv("children_living_hiv_df.csv", index_label="countries")